In [1]:
const name = "ItemSimilarity";

In [2]:
using CSV
using DataFrames
using LinearAlgebra
import NBInclude: @nbinclude
using SparseArrays
@nbinclude("../TrainingAlphas/Alpha.ipynb");

In [3]:
anime_to_uid = DataFrame(CSV.File("../../data/processed_data/anime_to_uid.csv"));
anime = DataFrame(CSV.File("../../data/raw_data/anime.csv", ntasks = 1));
anime_to_uid = innerjoin(anime_to_uid, anime, on = "anime_id");

In [184]:
W = read_params("NeuralExplicitItemCFUntuned")["m"][1].W;

[ Warning: 20220707 22:39:55 type Main.ItemCFLayer does not exist in workspace; reconstructing
[ Warning: 20220707 22:39:56 type Main.StorageLayer does not exist in workspace; reconstructing
[ Warning: 20220707 22:39:56 type Main.Hyperparams does not exist in workspace; reconstructing


In [186]:
W = read_params(read_params("ExplicitItemCF")["S"])["S"];

In [175]:
W = read_params("NeuralImplicitItemCFUntuned")["m"][1].W;

[ Warning: 20220707 22:39:08 type Main.ItemCFLayer does not exist in workspace; reconstructing
[ Warning: 20220707 22:39:09 type Main.StorageLayer does not exist in workspace; reconstructing
[ Warning: 20220707 22:39:09 type Main.Hyperparams does not exist in workspace; reconstructing


In [176]:
# m = read_params("NeuralExplicitAutoencoderUntuned")["m"]
# R = collect(m[end-1].weight')
# W = zeros(Float32, num_items(), num_items())
# @showprogress for i = 1:size(R)[2]
#     for j = 1:size(R)[2]
#         W[i, j] = norm(R[:, i] - R[:, j])
#     end
# end
# W = 1 ./ W;

In [177]:
for i = 1:size(W)[1]
    W[i, i] = Inf
end;

In [178]:
function get_neighborhood(item, W, K; transpose = false)
    if transpose
        weights = W[item, :]
    else
        weights = W[:, item]
    end
    order = sortperm(weights, rev = true)[1:K]
    order[1:K], weights[order[1:K]]
end;

In [179]:
function get_series(df)
    df = copy(df)
    df[!, :uid] = df[:, :anime] .- 1
    sim = innerjoin(df, anime_to_uid, on = "uid")
    sort(sim, :weight, rev = true)
end;

In [180]:
function display_related_series(mal_uid)
    uid = only(anime_to_uid[anime_to_uid.anime_id.==mal_uid, :uid])
    items, weights = get_neighborhood(uid + 1, W, 30)
    df = DataFrame(anime = items, weight = weights)
    get_series(df)
end;

In [181]:
ENV["LINES"] = 30;

In [185]:
display_related_series(9756)

,anime,weight,uid,anime_id,title
,Int64,Float32,Int64,Int64,String
1,18467,0.999868,18466,9756,Mahou Shoujo Madoka★Magica
2,7776,0.198524,7775,11981,Mahou Shoujo Madoka★Magica Movie 3: Hangyaku no Monogatari
3,14472,0.152269,14471,11979,Mahou Shoujo Madoka★Magica Movie 2: Eien no Monogatari
4,11666,0.148111,11665,11977,Mahou Shoujo Madoka★Magica Movie 1: Hajimari no Monogatari
5,18231,0.0888934,18230,5081,Bakemonogatari
6,18018,0.0870935,18017,17074,Monogatari Series: Second Season
7,282,0.0850711,281,11741,Fate/Zero 2nd Season
8,15702,0.0767691,15701,7311,Suzumiya Haruhi no Shoushitsu
9,2835,0.0756835,2834,10087,Fate/Zero
